#Install

In [1]:
%%time 

# Important library for many geopython libraries
!apt install gdal-bin python-gdal python3-gdal 
# Install rtree - Geopandas requirment
!apt install python3-rtree 
# Install Geopandas
!pip install git+git://github.com/geopandas/geopandas.git
# Install descartes - Geopandas requirment
!pip install descartes 
# Install Folium for Geographic data visualization
!pip install folium
# Install plotlyExpress
!pip install plotly_express
!pip install Shapely
!pip install pycopy-webbrowser

Reading package lists... Done
Building dependency tree       
Reading state information... Done
gdal-bin is already the newest version (2.2.3+dfsg-2).
python-gdal is already the newest version (2.2.3+dfsg-2).
The following additional packages will be installed:
  python3-numpy
Suggested packages:
  python-numpy-doc python3-nose python3-numpy-dbg
The following NEW packages will be installed:
  python3-gdal python3-numpy
0 upgraded, 2 newly installed, 0 to remove and 40 not upgraded.
Need to get 2,288 kB of archives.
After this operation, 13.2 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/main amd64 python3-numpy amd64 1:1.13.3-2ubuntu1 [1,943 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 python3-gdal amd64 2.2.3+dfsg-2 [346 kB]
Fetched 2,288 kB in 3s (877 kB/s)
Selecting previously unselected package python3-numpy.
(Reading database ... 148492 files and directories currently installed.)
Preparing to unpack .../python3-numpy_1%

In [2]:
from shapely.geometry import Point,Polygon
import pandas as pd
import webbrowser
import numpy as np
import geopandas as gpd
import matplotlib
import matplotlib.pyplot as plt 
import folium
from folium import plugins
from folium.plugins import HeatMap
import plotly_express as px
from google.colab import drive
from IPython.display import display
drive.mount('/content/drive')

Mounted at /content/drive


#Read

In [14]:
district_path =  "/content/drive/My Drive/2 Boundary/District_NE_new.geojson"
roadDB_path = "/content/drive/My Drive/2021 Route Prediction/Project-0/Road Data/roaddb.csv"

In [15]:
district_NE = gpd.read_file(district_path)
roadDB = pd.read_csv(roadDB_path)

#Simple data

In [ ]:
district_NE.head()

,AMP_CODE,PRV_CODE,AMP_NAME_T,AMP_NAME_E,Shape_Leng,Shape_Area,geometry
0,3001,30,เมืองนครราชสีมา,Mueang Nakhon Ratchasima,160600.306403,8.856562e+08,"POLYGON ((102.18341 15.13798, 102.18298 15.137..."
1,3002,30,ครบุรี,Khon Buri,261136.836300,2.168627e+09,"POLYGON ((102.15081 14.61461, 102.15115 14.614..."
2,3003,30,เสิงสาง,Soeng Sang,181829.710406,1.097496e+09,"POLYGON ((102.49675 14.50541, 102.49686 14.505..."
3,3004,30,คง,Khong,208115.123225,7.534105e+08,"POLYGON ((102.27233 15.53663, 102.27538 15.536..."
4,3005,30,บ้านเหลื่อม,Ban Lueam,88918.433064,2.569881e+08,"POLYGON ((102.12032 15.64497, 102.12072 15.644..."


In [ ]:
district_NE.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 321 entries, 0 to 320
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   AMP_CODE    321 non-null    object  
 1   PRV_CODE    321 non-null    object  
 2   AMP_NAME_T  321 non-null    object  
 3   AMP_NAME_E  321 non-null    object  
 4   Shape_Leng  321 non-null    float64 
 5   Shape_Area  321 non-null    float64 
 6   geometry    321 non-null    geometry
dtypes: float64(2), geometry(1), object(4)
memory usage: 17.7+ KB


In [ ]:
roadDB.head()

,rid,rd,km,ptype,lat,lon,latx,lonx,angle
0,1,3003,0.0,100m,14.492856,100.066108,14.493,100.066,180
1,1,3003,0.1,100m,14.493738,100.066290,14.494,100.066,180
2,1,3003,0.2,100m,14.494619,100.066476,14.495,100.066,179
3,1,3003,0.3,100m,14.495501,100.066658,14.496,100.067,179
4,1,3003,0.4,100m,14.496383,100.066839,14.496,100.067,179


In [ ]:
roadDB.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1093655 entries, 0 to 1093654
Data columns (total 9 columns):
 #   Column  Non-Null Count    Dtype  
---  ------  --------------    -----  
 0   rid     1093655 non-null  int64  
 1   rd      1093655 non-null  int64  
 2   km      1093655 non-null  float64
 3   ptype   1093655 non-null  object 
 4   lat     1093655 non-null  float64
 5   lon     1093655 non-null  float64
 6   latx    1093655 non-null  float64
 7   lonx    1093655 non-null  float64
 8   angle   1093655 non-null  int64  
dtypes: float64(5), int64(3), object(1)
memory usage: 75.1+ MB


#Algorithm

In [16]:
def chack_district(lon,lat):
  point = Point(lon,lat)
  for index in district_NE.index:
    if point.within(district_NE.loc[index,'geometry']):
      return district_NE.loc[index,'AMP_NAME_T']
  return 'false'

In [17]:
chack_district(102.433193,17.217769)

'เมืองหนองบัวลำภู'

#Road 1,2 digit

In [18]:
roadDB_digit = roadDB.loc[roadDB['rd']/10 < 10]

In [19]:
roadDB_digit.rd.unique()

array([32,  1,  3,  4, 35, 34, 31, 38,  9, 33, 11, 12, 41, 44, 22,  2, 37,
       42, 43, 23, 36, 24,  7, 21])

In [ ]:
roadDB_digit

,rid,rd,km,ptype,lat,lon,latx,lonx,angle
542625,3379,32,0.000,100m,14.170919,100.615903,14.171,100.616,180
542626,3379,32,0.100,100m,14.171708,100.616347,14.172,100.616,168
542627,3379,32,0.200,100m,14.172564,100.616619,14.173,100.617,159
542628,3379,32,0.300,100m,14.173457,100.616575,14.173,100.617,168
542629,3379,32,0.400,100m,14.174324,100.616331,14.174,100.616,178
...,...,...,...,...,...,...,...,...,...
1093090,6508,12,54.300,100m,16.689217,101.670215,16.689,101.670,180
1093091,6508,12,54.400,100m,16.688457,101.670716,16.688,101.671,180
1093092,6508,12,54.500,100m,16.687694,101.671213,16.688,101.671,180
1093093,6508,12,54.516,bend,16.687576,101.671292,16.688,101.671,33


#Filter RoadDB 1,2 digit

In [ ]:
for index in roadDB_digit.index:
  roadDB_digit.loc[index,'district'] = chack_district(roadDB_digit.loc[index,'lon'] , roadDB_digit.loc[index,'lat'])

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1596: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [ ]:
roadDB_digit

,rid,rd,km,ptype,lat,lon,latx,lonx,angle,district
542625,3379,32,0.000,100m,14.170919,100.615903,14.171,100.616,180,false
542626,3379,32,0.100,100m,14.171708,100.616347,14.172,100.616,168,false
542627,3379,32,0.200,100m,14.172564,100.616619,14.173,100.617,159,false
542628,3379,32,0.300,100m,14.173457,100.616575,14.173,100.617,168,false
542629,3379,32,0.400,100m,14.174324,100.616331,14.174,100.616,178,false
...,...,...,...,...,...,...,...,...,...,...
1093090,6508,12,54.300,100m,16.689217,101.670215,16.689,101.670,180,false
1093091,6508,12,54.400,100m,16.688457,101.670716,16.688,101.671,180,false
1093092,6508,12,54.500,100m,16.687694,101.671213,16.688,101.671,180,false
1093093,6508,12,54.516,bend,16.687576,101.671292,16.688,101.671,33,false


In [ ]:
len(roadDB_digit.loc[roadDB_digit['district'] == 'false'])

63441

In [ ]:
for index in roadDB_digit.index:
  if roadDB_digit.loc[index,'district'] == 'false':
    roadDB_digit.drop(index,axis=0,inplace=True)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [ ]:
roadDB_digit

,rid,rd,km,ptype,lat,lon,latx,lonx,angle,district
715238,4489,22,0.0,100m,17.321291,104.416470,17.321,104.416,180,กุสุมาลย์
715239,4489,22,0.1,100m,17.321363,104.417409,17.321,104.417,179,เมืองนครพนม
715240,4489,22,0.2,100m,17.321415,104.418349,17.321,104.418,176,เมืองนครพนม
715241,4489,22,0.3,100m,17.321394,104.419290,17.321,104.419,172,เมืองนครพนม
715242,4489,22,0.4,100m,17.321241,104.420215,17.321,104.420,164,เมืองนครพนม
...,...,...,...,...,...,...,...,...,...,...
1092985,6508,12,43.8,100m,16.718843,101.591987,16.719,101.592,179,คอนสาร
1093003,6508,12,45.6,100m,16.707472,101.603346,16.707,101.603,180,คอนสาร
1093004,6508,12,45.7,100m,16.707015,101.604155,16.707,101.604,178,คอนสาร
1093005,6508,12,45.8,100m,16.706528,101.604944,16.707,101.605,170,คอนสาร


#Save file

In [ ]:
path = "/content/drive/My Drive/2 Boundary/roadDB_NE.csv"

In [ ]:
roadDB_digit.to_csv(path)

#Plot road 1 ,2 digit

In [ ]:
lat = roadDB_digit.lat
lon = roadDB_digit.lon
coord = pd.DataFrame()

In [ ]:
coord.insert(0,"lat",lat)
coord.insert(1,"lon",lon)

In [ ]:
coord.reset_index(inplace=True)

In [ ]:
heat_data = [[row['lat'],row['lon']] for index, row in coord.iterrows()]

In [ ]:
mapit = folium.Map( location=[ 17.87914890447798,102.7390305957055], zoom_start=6 )
HeatMap(heat_data).add_to(mapit)
display(mapit)

In [5]:
path_roadDB_NE = "/content/drive/My Drive/2 Boundary/roadDB_NE.csv"

In [6]:
roadDB_NE = pd.read_csv(path_roadDB_NE)

In [8]:
roadDB_NE.rd.unique()

array([22, 12,  2, 23, 24, 21])

#Road 3 digit

In [40]:
roadDB_3digit = roadDB.loc[roadDB['rd']/100 < 10]

In [41]:
roadDB_3digit

,rid,rd,km,ptype,lat,lon,latx,lonx,angle
539774,3344,354,0.000,100m,13.665967,100.719511,13.666,100.720,180
539775,3344,354,0.100,100m,13.665919,100.720435,13.666,100.720,180
539776,3344,354,0.200,100m,13.665870,100.721359,13.666,100.721,180
539777,3344,354,0.300,100m,13.665822,100.722283,13.666,100.722,180
539778,3344,354,0.400,100m,13.665774,100.723207,13.666,100.723,180
...,...,...,...,...,...,...,...,...,...
1093090,6508,12,54.300,100m,16.689217,101.670215,16.689,101.670,180
1093091,6508,12,54.400,100m,16.688457,101.670716,16.688,101.671,180
1093092,6508,12,54.500,100m,16.687694,101.671213,16.688,101.671,180
1093093,6508,12,54.516,bend,16.687576,101.671292,16.688,101.671,33


In [42]:
roadDB_3digit = roadDB_3digit.loc[roadDB_3digit['rd']/100 > 1]

In [44]:
roadDB_3digit.rd.unique()

array([354, 347, 303, 308, 307, 351, 306, 340, 352, 370, 346, 304, 309,
       356, 338, 301, 305, 302, 336, 345, 341, 350, 114, 107, 103, 108,
       116, 106, 128, 105, 118, 120, 121, 127, 109, 323, 334, 324, 333,
       322, 368, 367, 321, 311, 357, 104, 101, 112, 115, 401, 404, 420,
       403, 408, 415, 417, 419, 241, 213, 212, 238, 227, 222, 240, 211,
       243, 242, 233, 239, 295, 223, 123, 129, 375, 327, 325, 326, 330,
       410, 414, 407, 409, 406, 418, 416, 228, 205, 216, 229, 225, 201,
       210, 208, 202, 230, 365, 315, 316, 349, 319, 317, 363, 344, 314,
       364, 331, 361, 332, 359, 348, 207, 224, 226, 218, 204, 320, 219,
       288, 372, 206, 290, 215, 232, 291, 214, 292, 117, 111, 102, 113,
       126, 125, 402, 412, 362, 122, 369, 335, 310, 366, 220, 217, 294,
       221, 293, 231, 234])

#Filter RoadDB 3 digit 

In [45]:
for index in roadDB_3digit.index:
  roadDB_3digit.loc[index,'district'] = chack_district(roadDB_3digit.loc[index,'lon'] , roadDB_3digit.loc[index,'lat'])

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1596: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [46]:
roadDB_3digit

,rid,rd,km,ptype,lat,lon,latx,lonx,angle,district
539774,3344,354,0.000,100m,13.665967,100.719511,13.666,100.720,180,false
539775,3344,354,0.100,100m,13.665919,100.720435,13.666,100.720,180,false
539776,3344,354,0.200,100m,13.665870,100.721359,13.666,100.721,180,false
539777,3344,354,0.300,100m,13.665822,100.722283,13.666,100.722,180,false
539778,3344,354,0.400,100m,13.665774,100.723207,13.666,100.723,180,false
...,...,...,...,...,...,...,...,...,...,...
1092169,6504,210,34.300,100m,17.223777,102.413920,17.224,102.414,179,เมืองหนองบัวลำภู
1092170,6504,210,34.400,100m,17.224231,102.413108,17.224,102.413,179,เมืองหนองบัวลำภู
1092171,6504,210,34.500,100m,17.224669,102.412286,17.225,102.412,180,เมืองหนองบัวลำภู
1092172,6504,210,34.546,bend,17.224873,102.411912,17.225,102.412,118,เมืองหนองบัวลำภู


In [48]:
len(roadDB_3digit.loc[roadDB_3digit['district'] == 'false'])

76305

In [49]:
for index in roadDB_3digit.index:
  if roadDB_3digit.loc[index,'district'] == 'false':
    roadDB_3digit.drop(index,axis=0,inplace=True)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [50]:
roadDB_3digit

,rid,rd,km,ptype,lat,lon,latx,lonx,angle,district
707734,4459,241,0.000,100m,17.191885,104.095429,17.192,104.095,180,เมืองสกลนคร
707735,4459,241,0.100,100m,17.191096,104.094976,17.191,104.095,179,เมืองสกลนคร
707736,4459,241,0.200,100m,17.190311,104.094519,17.190,104.095,179,เมืองสกลนคร
707737,4459,241,0.300,100m,17.189527,104.094057,17.190,104.094,180,เมืองสกลนคร
707738,4459,241,0.400,100m,17.188737,104.093607,17.189,104.094,179,เมืองสกลนคร
...,...,...,...,...,...,...,...,...,...,...
1092169,6504,210,34.300,100m,17.223777,102.413920,17.224,102.414,179,เมืองหนองบัวลำภู
1092170,6504,210,34.400,100m,17.224231,102.413108,17.224,102.413,179,เมืองหนองบัวลำภู
1092171,6504,210,34.500,100m,17.224669,102.412286,17.225,102.412,180,เมืองหนองบัวลำภู
1092172,6504,210,34.546,bend,17.224873,102.411912,17.225,102.412,118,เมืองหนองบัวลำภู


In [51]:
path2 = "/content/drive/My Drive/2 Boundary/roadDB_NE_3digit.csv"
roadDB_3digit.to_csv(path2)

In [54]:
roadDB_3digit.rd.unique()

array([241, 213, 212, 238, 227, 222, 240, 211, 243, 242, 233, 239, 295,
       223, 228, 205, 216, 229, 225, 201, 210, 208, 202, 230, 207, 224,
       304, 226, 218, 204, 219, 288, 206, 290, 348, 215, 232, 291, 214,
       292, 220, 217, 294, 221, 293, 231])

#plot road 3 digit

In [52]:
lat2 = roadDB_3digit.lat
lon2 = roadDB_3digit.lon
coord2 = pd.DataFrame()
coord2.insert(0,"lat",lat2)
coord2.insert(1,"lon",lon2)
coord2.reset_index(inplace=True)
heat_data2 = [[row['lat'],row['lon']] for index, row in coord2.iterrows()]

In [53]:
mapit2 = folium.Map( location=[ 17.87914890447798,102.7390305957055], zoom_start=6 )
HeatMap(heat_data2).add_to(mapit2)
display(mapit2)